In [1]:
# Import Libraries 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import datetime as dt
import time

In [10]:
# Open Chrome and full screen
browser = webdriver.Chrome()
browser.maximize_window()

In [11]:
# Open World Weather Online 
browser.get('https://www.worldweatheronline.com/ho-chi-minh-city-weather-history/vn.aspx')

In [12]:
# Crawl summarize data of each day.
def crawl_data_years(date_query):
    day = date_query.day
    month = date_query.month
    year = date_query.year

    # Query by date
    browser.find_element_by_id('ctl00_MainContentHolder_txtPastDate').clear()
    browser.find_element_by_id('ctl00_MainContentHolder_txtPastDate').send_keys("%02d/%02d/%04d"%(month,day,year))
    browser.find_element_by_id('ctl00_MainContentHolder_butShowPastWeather').click()

    # Sleep
    time.sleep(1)

    # Crawl data
    inf = browser.find_elements_by_class_name("text-center")[4].text.split("\n")
    inf_matrix = np.reshape(inf[9:], (12, 10))
    
    # Save to dataframe
    weathers = pd.DataFrame(inf_matrix, columns = ['Date', 'Weather', 'Max', 'Min', 'Wind', 'Direction', 'Rain', 'Humidity', 'Cloud', 'Pressure'])
    weathers['Date'] = [dt.datetime(int(y), month, day) for y in weathers.Date]
    return weathers

In [16]:
# Crawl all information of each day (includes many times in a day)
def crawl_data_each_day(date_query):
    day = date_query.day
    month = date_query.month
    year = date_query.year

    # Query by date
    browser.find_element_by_id('ctl00_MainContentHolder_txtPastDate').clear()
    browser.find_element_by_id('ctl00_MainContentHolder_txtPastDate').send_keys("%02d/%02d/%04d"%(month,day,year))
    browser.find_element_by_id('ctl00_MainContentHolder_butShowPastWeather').click()

    # Sleep
    time.sleep(1)

    # Crawl data
    inf = browser.find_elements_by_class_name("text-center")[1].text.split("\n")
    inf_matrix = np.reshape(inf[11:], (8, 11))
    
    # Save to dataframe
    weathers = pd.DataFrame(inf_matrix, columns = ['Time', 'Weather', 'Temp', 'Feels', 'Wind', 'Gust', 'Rain', 'Humidity', 'Cloud', 'Pressure', 'Vis'])
    weathers['Time'] = [dt.datetime.strptime("%04d-%02d-%02d %s"%(year, month, day, time),'%Y-%m-%d %H:%M') for time in weathers.Time]
    return weathers

In [ ]:
weather_df = pd.DataFrame(columns = ['Date', 'Weather', 'Max', 'Min', 'Wind', 'Direction', 'Rain', 'Humidity', 'Cloud', 'Pressure'])
date_query = dt.datetime(2020,1,1) 
for i in range(355):
    if date_query != dt.datetime(2020,2,29):
        weathers = crawl_data_years(date_query)
        weather_df = weather_df.append(weathers, ignore_index=True)
    date_query = date_query + dt.timedelta(1)

date_query = dt.datetime(2019,12,21) 
for i in range(11):
    if date_query != dt.datetime(2020,2,29):
        weathers = crawl_data_years(date_query)
        weather_df = weather_df.append(weathers, ignore_index=True)
    date_query = date_query + dt.timedelta(1)

In [29]:
weather_df2 = pd.DataFrame(columns = ['Time', 'Weather', 'Temp', 'Feels', 'Wind', 'Gust', 'Rain', 'Humidity', 'Cloud', 'Pressure', 'Vis'])
date_query = dt.datetime(2009,1,1) 
for i in range(4372):
    weathers = crawl_data_each_day(date_query)
    weather_df2 = weather_df2.append(weathers, ignore_index=True)
    date_query = date_query + dt.timedelta(1)

In [30]:
weather_df2

,Time,Weather,Temp,Feels,Wind,Gust,Rain,Humidity,Cloud,Pressure,Vis
0,2009-01-01 00:00:00,Fog,23 °c,25 °c,9 km/h from NNW,15 km/h,0.0 mm,97%,100%,1010 mb,Poor
1,2009-01-01 03:00:00,Light drizzle,22 °c,25 °c,9 km/h from NNW,13 km/h,0.4 mm,97%,84%,1010 mb,Poor
2,2009-01-01 06:00:00,Fog,22 °c,25 °c,6 km/h from N,8 km/h,0.0 mm,98%,100%,1011 mb,Poor
3,2009-01-01 09:00:00,Cloudy,27 °c,31 °c,6 km/h from NNE,7 km/h,0.1 mm,83%,64%,1011 mb,Excellent
4,2009-01-01 12:00:00,Partly cloudy,28 °c,34 °c,3 km/h from NE,3 km/h,0.0 mm,76%,62%,1010 mb,Excellent
...,...,...,...,...,...,...,...,...,...,...,...
34971,2020-12-20 09:00:00,Partly cloudy,26 °c,28 °c,17 km/h from NNE,22 km/h,0.0 mm,64%,14%,1012 mb,Excellent
34972,2020-12-20 12:00:00,Partly cloudy,30 °c,31 °c,17 km/h from NNE,19 km/h,0.0 mm,54%,13%,1010 mb,Excellent
34973,2020-12-20 15:00:00,Partly cloudy,31 °c,33 °c,14 km/h from WSW,18 km/h,0.0 mm,55%,12%,1008 mb,Excellent
34974,2020-12-20 18:00:00,Partly cloudy,28 °c,30 °c,15 km/h from SE,24 km/h,0.0 mm,60%,18%,1009 mb,Excellent


In [31]:
weather_df2.to_csv("weather_full_df.csv", index = False)

In [17]:
weather_df

,Date,Weather,Max,Min,Wind,Direction,Rain,Humidity,Cloud,Pressure
0,2009-01-01,Moderate or heavy rain shower,28 °c,22 °c,6 km/h,SE,5.2 mm,89%,74%,1010 mb
1,2010-01-01,Sunny,33 °c,23 °c,5 km/h,E,0.0 mm,60%,18%,1010 mb
2,2011-01-01,Cloudy,29 °c,23 °c,4 km/h,S,0.0 mm,81%,41%,1008 mb
3,2012-01-01,Patchy rain possible,32 °c,23 °c,7 km/h,E,1.0 mm,76%,40%,1012 mb
4,2013-01-01,Partly cloudy,32 °c,23 °c,4 km/h,SE,0.0 mm,69%,15%,1010 mb
...,...,...,...,...,...,...,...,...,...,...
4364,2015-12-31,Partly cloudy,33 °c,23 °c,12 km/h,ENE,0.0 mm,57%,23%,1014 mb
4365,2016-12-31,Patchy rain possible,31 °c,23 °c,9 km/h,NE,1.4 mm,74%,27%,1012 mb
4366,2017-12-31,Sunny,32 °c,23 °c,9 km/h,ENE,0.0 mm,60%,11%,1011 mb
4367,2018-12-31,Cloudy,32 °c,23 °c,13 km/h,ENE,0.7 mm,62%,30%,1013 mb


In [ ]:
weather_df.to_csv("weather.csv", index = False)